In [1]:
# ===== Import necessary libraries =====
import cv2
import numpy as np
import os, sys
import dlib
import glob
from imutils import face_utils
import face_recognition
from skimage import io

# ===== Initialize varialbes =====
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
face_detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
face_recognition_model = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')

count = 1
#face_data = []
dataset_path = './knn_dlib/'
offset = 10

# ========== 1) capture image ==========
webcam = cv2.VideoCapture(0)
# ===== Ask user to enter name to create his/her image folder =====
file_name = input("Enter the name of the person :  ")
# ===== capture 30 images =====
while (webcam.isOpened() and count <= 30) :
    ret,frame = webcam.read()
    if(ret == False):
        continue
    # ===== convert to gray frame =====
    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)    
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    faces = sorted(faces, key=lambda f:f[2]*f[3])    # it was used in openCV -> not sure about deleting it
    for face in faces : 
        x, y, w, h =  face 
        # ===== draw bounding box around face =====
        #cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
    # ===== extract only face in different window =====
    face_section = frame[y-offset:y+h+offset, x-offset:x+w+offset]
    face_section = cv2.resize(face_section,(150, 150))
    print('face section', face_section)
    # ===== create file_name folder under knn_dlib =====
    if not os.path.exists(dataset_path + "/" + file_name):
        os.makedirs(dataset_path + "/" + file_name)
    # ===== save key image as frame size =====
    #fileName = dataset_path + "/" + file_name + "/" + file_name + "_" + str(count) + ".jpg"
    #cv2.imwrite(fileName, frame)
    # ===== save key image as face_section size =====
    fileName = dataset_path + "/" + file_name + "/" + file_name + "_" + str(count) + ".jpg"
    cv2.imwrite(fileName, face_section)
    # ===== increment count =====
    count += 1
    print('count', count)
    # ===== Display both window =====
    cv2.imshow("FACE CROP",face_section) 
    cv2.imshow("CAPTURE IMG",frame)
    # ===== Hit 'q' to QUIT =====
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
# ===== Reset count to 1 & close webcam + window =====
#count = 1
webcam.release()
cv2.destroyAllWindows()        

# ========== 2) load image and save its info as a key to compare ==========
def get_face_encodings(face):
    """
        return np.array of face recognition model which contains location, landmarks for face encoding
    """
    bounds = face_detector(face, 1) # detect face rectangles 
    #print('bounds ', bounds)
    faces_landmarks = [shape_predictor(face, face_bounds) for face_bounds in bounds]
    #print('face_landmarks ', faces_landmarks)
    return [np.array(face_recognition_model.compute_face_descriptor(face, face_pose, 1)) for face_pose in faces_landmarks]

def get_face_matches(known_faces, face):
    """
        return euclidean distance 
    """
    #print('known face - face ', known_faces-face)
    return np.linalg.norm(known_faces - face, axis=1)

def find_match(known_faces, person_names, face):
    """
        min distance is the best prediction 
    """
    matches = get_face_matches(known_faces, face) 
    print('matches ', matches)
    min_index = matches.argmin() # min distance index
    print('min index ', min_index)
    min_value = matches[min_index] # min distance
    print('min value ', min_value)
    matchPercent = 100 - (min_value * 100) # convert to percentage
    print('matchPercent ', matchPercent)
    if matchPercent >= 80 : # at least 80% of correction 
        return person_names[min_index] +" {0:.2f}%".format(matchPercent)
    return 'Not Found'

def load_face_encodings(faces_folder_path):
    """
        Load face images in person's name folder in a separate window 
    """
    image_filenames = filter(lambda x: x.endswith('.jpg'), os.listdir(faces_folder_path))
    image_filenames = sorted(image_filenames)
    #person_names = [x[:-4] for x in image_filenames] # exclude .jpg 
    person_names = []
    for x in image_filenames :
        #print('image file name ', x)
        index = x.find('_')
        person_names.append(x[:index]) # exclude from '_'

    full_paths_to_images = [faces_folder_path + x for x in image_filenames]
    face_encodings = []

    win = dlib.image_window()

    for path_to_image in full_paths_to_images:
        face = io.imread(path_to_image)

        faces_bounds = face_detector(face, 1)

        if len(faces_bounds) != 1:
            print("Expected one and only one face per image: " + path_to_image + " - it has " + str(len(faces_bounds)))
            exit()

        face_bounds = faces_bounds[0]
        face_landmarks = shape_predictor(face, face_bounds)
        face_encoding = np.array(face_recognition_model.compute_face_descriptor(face, face_landmarks, 1))

        win.clear_overlay()
        win.set_title("Face Encoding")
        win.set_image(face)
        win.add_overlay(face_bounds)
        win.add_overlay(face_landmarks)
        face_encodings.append(face_encoding)
    #print('face encoding result ', face_encoding)
    return face_encodings, person_names

# ========== 3) initialize webcam to compare key image features ==========

data_dir = os.path.expanduser('./knn_dlib')
faces_folder_path = data_dir + '/hyFS/'  # just need to change folder's name to encode 
#faces_folder_path = data_dir + '/hy/'  # just need to change folder's name to encode 
#faces_folder_path = data_dir + '/sk/'  # just need to change folder's name to encode 
#faces_folder_path = data_dir + '/skFS/'  # just need to change folder's name to encode 
face_encodings, person_names = load_face_encodings(faces_folder_path)

face_data = []
        
camera = cv2.VideoCapture(0)
old_faces = []
while True:
    ret, frame = camera.read()
    if not ret:
        break

    frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)    
    faces = face_detector(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_rects = face_cascade.detectMultiScale(gray, scaleFactor = 1.3, minNeighbors = 5, minSize = (50, 50), flags = cv2.CASCADE_SCALE_IMAGE)
    
    if len(old_faces) < len(faces):
        old_faces = []
        for face in faces:
            tracker = dlib.correlation_tracker()
            tracker.start_track(frame, face)
            old_faces.append(tracker)
    else:
        for i, tracker in enumerate(old_faces):
            quality = tracker.update(frame)
            if quality > 7:
                pos = tracker.get_position()
                pos = dlib.rectangle(int(pos.left()), int(pos.top()), int(pos.right()), int(pos.bottom()))
                face = frame[pos.top():pos.top() + pos.bottom(), pos.left():pos.left() + pos.right()]
                face_encodings_in_image = get_face_encodings(face)
                if (face_encodings_in_image) :
                    match = find_match(face_encodings, person_names, face_encodings_in_image[0])
                    cv2.putText(frame, match, (pos.left()-50, pos.top()-15), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
                else : 
                    cv2.putText(frame, 'Unknown', (pos.left()-15, pos.top()-15), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
                cv2.rectangle(frame, (pos.left(), pos.top()), (pos.right(), pos.bottom()), (0, 255, 255), 2)
            else:
                old_faces.pop(i)
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
camera.release()
cv2.destroyAllWindows()


Enter the name of the person :  a
face section [[[204 221 224]
  [204 220 223]
  [205 220 224]
  ...
  [177 199 202]
  [174 200 204]
  [168 198 212]]

 [[205 221 223]
  [204 219 222]
  [205 220 224]
  ...
  [181 200 205]
  [175 199 203]
  [167 197 210]]

 [[205 221 222]
  [205 221 222]
  [206 222 223]
  ...
  [179 198 203]
  [172 198 202]
  [165 196 210]]

 ...

 [[101  92  74]
  [109  99  82]
  [115 106  86]
  ...
  [ 51  49  49]
  [ 51  48  51]
  [ 48  45  50]]

 [[102  90  75]
  [110 100  84]
  [114 104  87]
  ...
  [ 50  51  48]
  [ 53  52  53]
  [ 50  50  54]]

 [[101  90  75]
  [109  99  84]
  [118 108  90]
  ...
  [ 49  49  49]
  [ 51  52  53]
  [ 48  50  54]]]
count 2
face section [[[205 218 221]
  [206 219 222]
  [206 219 221]
  ...
  [ 57  72  87]
  [ 83  83  76]
  [ 88  87  75]]

 [[205 219 222]
  [206 219 222]
  [206 219 222]
  ...
  [ 52  69  85]
  [ 84  83  77]
  [ 91  87  76]]

 [[204 219 222]
  [205 220 224]
  [205 220 224]
  ...
  [ 50  69  85]
  [ 84  82  78]
  [ 90  

face section [[[189 207 210]
  [189 207 210]
  [191 206 210]
  ...
  [179 194 197]
  [178 194 196]
  [176 192 195]]

 [[191 206 210]
  [194 209 212]
  [194 209 212]
  ...
  [181 195 198]
  [177 193 196]
  [177 192 196]]

 [[195 209 215]
  [191 208 214]
  [191 208 214]
  ...
  [180 195 198]
  [178 194 197]
  [176 192 196]]

 ...

 [[112 104  79]
  [112 104  79]
  [106  98  73]
  ...
  [ 29  37  50]
  [ 28  35  52]
  [ 26  34  54]]

 [[114 103  79]
  [111 101  76]
  [107  97  71]
  ...
  [ 31  38  52]
  [ 27  33  52]
  [ 26  34  55]]

 [[114 106  82]
  [111 103  78]
  [108  97  73]
  ...
  [ 32  38  52]
  [ 25  33  49]
  [ 26  34  54]]]
count 17
face section [[[191 206 211]
  [194 208 214]
  [191 206 212]
  ...
  [178 194 197]
  [178 193 198]
  [177 192 198]]

 [[191 206 211]
  [191 206 210]
  [194 209 212]
  ...
  [178 194 197]
  [178 194 197]
  [177 192 195]]

 [[191 206 210]
  [191 206 210]
  [194 209 212]
  ...
  [179 196 199]
  [180 195 198]
  [178 194 197]]

 ...

 [[109 101  86]
 

face section [[[191 206 211]
  [192 207 213]
  [195 210 214]
  ...
  [175 191 196]
  [175 191 196]
  [172 190 195]]

 [[191 206 211]
  [194 207 213]
  [195 209 215]
  ...
  [172 189 194]
  [174 191 196]
  [171 189 196]]

 [[193 206 211]
  [193 206 211]
  [196 209 212]
  ...
  [174 191 198]
  [172 190 197]
  [168 187 195]]

 ...

 [[110 102  85]
  [108 101  82]
  [107  98  78]
  ...
  [ 29  36  56]
  [ 24  30  53]
  [ 24  31  57]]

 [[113 105  85]
  [111 101  82]
  [108  97  75]
  ...
  [ 26  33  50]
  [ 23  29  51]
  [ 23  30  56]]

 [[115 107  85]
  [112 103  81]
  [110  98  77]
  ...
  [ 26  33  50]
  [ 23  30  51]
  [ 22  28  53]]]
count 30
face section [[[193 209 212]
  [194 210 213]
  [196 211 214]
  ...
  [176 192 197]
  [174 191 196]
  [174 191 196]]

 [[192 209 213]
  [192 209 212]
  [196 211 214]
  ...
  [177 193 196]
  [174 192 194]
  [174 191 196]]

 [[190 208 211]
  [195 210 213]
  [194 209 212]
  ...
  [176 191 196]
  [173 190 195]
  [171 190 195]]

 ...

 [[109 104  84]
 